In [2]:
import pandas as pd
import numpy as np

In [3]:
data=pd.read_csv("Stars.csv")
data

,Temperature,L,R,A_M,Color,Spectral_Class,Type
0,3068,0.002400,0.1700,16.12,Red,M,0
1,3042,0.000500,0.1542,16.60,Red,M,0
2,2600,0.000300,0.1020,18.70,Red,M,0
3,2800,0.000200,0.1600,16.65,Red,M,0
4,1939,0.000138,0.1030,20.06,Red,M,0
...,...,...,...,...,...,...,...
235,38940,374830.000000,1356.0000,-9.93,Blue,O,5
236,30839,834042.000000,1194.0000,-10.63,Blue,O,5
237,8829,537493.000000,1423.0000,-10.73,White,A,5
238,9235,404940.000000,1112.0000,-11.23,White,A,5


In [4]:
selectrandom=np.random.randint(0,240,size=10)
selectrandom

array([119,  87,  44, 235,  13, 212, 153,  79, 171,  31])

In [5]:
newTemp=data['Temperature'].copy()
newTemp[list(selectrandom)]=np.nan
data['newTemp']=newTemp
data

,Temperature,L,R,A_M,Color,Spectral_Class,Type,newTemp
0,3068,0.002400,0.1700,16.12,Red,M,0,3068.0
1,3042,0.000500,0.1542,16.60,Red,M,0,3042.0
2,2600,0.000300,0.1020,18.70,Red,M,0,2600.0
3,2800,0.000200,0.1600,16.65,Red,M,0,2800.0
4,1939,0.000138,0.1030,20.06,Red,M,0,1939.0
...,...,...,...,...,...,...,...,...
235,38940,374830.000000,1356.0000,-9.93,Blue,O,5,NaN
236,30839,834042.000000,1194.0000,-10.63,Blue,O,5,30839.0
237,8829,537493.000000,1423.0000,-10.73,White,A,5,8829.0
238,9235,404940.000000,1112.0000,-11.23,White,A,5,9235.0


In [6]:
data2=data.loc[data['newTemp']==data['Temperature']]

In [7]:
data2

,Temperature,L,R,A_M,Color,Spectral_Class,Type,newTemp
0,3068,0.002400,0.1700,16.12,Red,M,0,3068.0
1,3042,0.000500,0.1542,16.60,Red,M,0,3042.0
2,2600,0.000300,0.1020,18.70,Red,M,0,2600.0
3,2800,0.000200,0.1600,16.65,Red,M,0,2800.0
4,1939,0.000138,0.1030,20.06,Red,M,0,1939.0
...,...,...,...,...,...,...,...,...
234,21904,748490.000000,1130.0000,-7.67,Blue-white,B,5,21904.0
236,30839,834042.000000,1194.0000,-10.63,Blue,O,5,30839.0
237,8829,537493.000000,1423.0000,-10.73,White,A,5,8829.0
238,9235,404940.000000,1112.0000,-11.23,White,A,5,9235.0


In [8]:
LR=data2[['L','R']]
np.array([np.array(LR['L']),np.array(LR['R'])])
arr=np.array([[x,y]for x,y in zip(np.array(LR['L']),np.array(LR['R']))])
arr

array([[2.40000e-03, 1.70000e-01],
       [5.00000e-04, 1.54200e-01],
       [3.00000e-04, 1.02000e-01],
       [2.00000e-04, 1.60000e-01],
       [1.38000e-04, 1.03000e-01],
       [6.50000e-04, 1.10000e-01],
       [7.30000e-04, 1.27000e-01],
       [4.00000e-04, 9.60000e-02],
       [6.90000e-04, 1.10000e-01],
       [1.80000e-04, 1.30000e-01],
       [2.90000e-03, 5.10000e-01],
       [1.22000e-02, 3.76100e-01],
       [4.00000e-04, 1.96000e-01],
       [6.00000e-04, 1.40000e-01],
       [3.80000e-03, 2.40000e-01],
       [1.80000e-03, 1.60000e-01],
       [3.67000e-03, 4.70000e-01],
       [3.62000e-03, 1.96700e-01],
       [3.90000e-02, 3.51000e-01],
       [5.60000e-02, 8.40000e-03],
       [4.90000e-04, 1.23400e-02],
       [1.70000e-04, 1.10000e-02],
       [5.00000e-04, 1.00000e-02],
       [1.30000e-02, 1.40000e-02],
       [8.50000e-05, 9.84000e-03],
       [8.10000e-04, 9.70000e-03],
       [1.10000e-04, 1.28000e-02],
       [1.50000e-04, 1.10000e-02],
       [8.00000e-05,

In [9]:
from sklearn.linear_model import LinearRegression

In [10]:
reg=LinearRegression().fit(arr,data2['newTemp'])

In [11]:
data4=data.loc[data['newTemp']!=data['Temperature']]
data4

,Temperature,L,R,A_M,Color,Spectral_Class,Type,newTemp
13,3628,0.00550,0.3930,10.48,Red,M,1,NaN
31,30000,28840.00000,6.3000,-4.20,Blue-white,B,3,NaN
44,3008,280000.00000,25.0000,-6.00,Red,M,4,NaN
79,3158,0.00135,0.1610,13.98,Red,M,1,NaN
87,10980,0.00074,0.0087,11.19,Blue White,B,2,NaN
119,3780,200000.00000,1324.0000,-10.70,Red,M,5,NaN
153,16390,1278.00000,5.6800,-3.32,Blue-white,B,3,NaN
171,3500,138000.00000,1420.0000,-8.18,Red,M,5,NaN
212,13089,788.00000,5.9920,-0.12,Blue-white,A,3,NaN
235,38940,374830.00000,1356.0000,-9.93,Blue,O,5,NaN


In [12]:
reg.predict([np.array([174000.00000,1284.0000])])

array([8011.82350865])

In [13]:
data4LR=data4[['L','R']]
arr4=np.array([[x,y]for x,y in zip(np.array(data4LR['L']),np.array(data4LR['R']))])
data4['newTemp']=reg.predict(arr4)

C:\Users\Mosp\AppData\Local\Temp/ipykernel_17848/2809870597.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data4['newTemp']=reg.predict(arr4)


In [14]:
data4

,Temperature,L,R,A_M,Color,Spectral_Class,Type,newTemp
13,3628,0.00550,0.3930,10.48,Red,M,1,8444.109882
31,30000,28840.00000,6.3000,-4.20,Blue-white,B,3,9190.236948
44,3008,280000.00000,25.0000,-6.00,Red,M,4,15817.511917
79,3158,0.00135,0.1610,13.98,Red,M,1,8445.026991
87,10980,0.00074,0.0087,11.19,Blue White,B,2,8445.629098
119,3780,200000.00000,1324.0000,-10.70,Red,M,5,8547.388764
153,16390,1278.00000,5.6800,-3.32,Blue-white,B,3,8457.305748
171,3500,138000.00000,1420.0000,-8.18,Red,M,5,6513.625953
212,13089,788.00000,5.9920,-0.12,Blue-white,A,3,8442.998543
235,38940,374830.00000,1356.0000,-9.93,Blue,O,5,13085.519140


In [15]:
data['newTemp'].loc[data['Temperature']!=data['newTemp']]=data4['newTemp']

C:\Users\Mosp\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [16]:
data[['Temperature','newTemp']].loc[data['Temperature']!=data['newTemp']]

,Temperature,newTemp
13,3628,8444.109882
31,30000,9190.236948
44,3008,15817.511917
79,3158,8445.026991
87,10980,8445.629098
119,3780,8547.388764
153,16390,8457.305748
171,3500,6513.625953
212,13089,8442.998543
235,38940,13085.519140


In [19]:
from scipy.stats import chi2_contingency
from sklearn.preprocessing import LabelEncoder


In [20]:
color=data['Color']
lbl = LabelEncoder()
color=lbl.fit_transform(color)

Spectral_Class=data['Spectral_Class']
lbl = LabelEncoder()
Spectral_Class=lbl.fit_transform(Spectral_Class)


In [36]:
df=(len(np.unique(Spectral_Class))-1)*(len(np.unique(color))-1)
df

96

In [40]:
x_squared, p, dof, expected = chi2_contingency([Spectral_Class,color])
x_squared

568.4638691525676

In [42]:
from scipy.stats import chi2
prob=0.95
critical = chi2.ppf(prob, df)
critical

119.87093929856714